# Sample alignment and coaddition workflow for KCWI data cubes

In [3]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import glob
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from kcwi_jnb import transform
from kcwi_jnb import combine
from kcwi_jnb.cube import DataCube
#from importlib import reload
#reload(transform)



## Grab lists of intensity and variance cubes

In [11]:
icubed = glob.glob('*icubed.fits')
vcubed = glob.glob('*vcubed.fits')

## Identify brightest pixel in white light image of each exposure and set coords equal across exposures
### Creates directories for white light images and corrected cubes ('nb_off' and 'corrCubes' by default)

In [12]:
brightcoords = transform.white_light_offset(icubed,slicer='M',varcubelist=vcubed)

## Project cubes aligned to brightest pixel in larger space
### May not be necessary if observations were not dithered

In [13]:
icubed_corr =glob.glob('corrCubes/*icubed.fits')
vcubed_corr =glob.glob('corrCubes/*vcubed.fits')
cubestack,stackwcs = transform.align_cubes(icubed_corr,brightcoords)
varstack,varstackwcs = transform.align_cubes(vcubed_corr,brightcoords)

## Perform the coaddition using the weighted mean (inverse variance weights)

In [14]:
combine.weight_mean_coadd(cubestack,varstack,stackwcs,varstackwcs,outfile='corrCubes/weightedMean_coadd_flux.fits',
                         outvarfile='corrCubes/weightedMean_coadd_variance.fits')

133.01872
133.2611
131.86873
129.35011
129.32794
124.89285


## Coadd using a simple median
### May be useful if some issue with the variance cubes

In [8]:
combine.median_coadd(cubestack,stackwcs,outfile='corrCubes/median_coadd_flux.fits')

/Users/jnburchett/.pyenv/versions/anaconda2-5.0.0/lib/python2.7/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median for 2283886 results
  r = func(a, **kwargs)
/Users/jnburchett/Dropbox/Code/linetools/linetools/spectra/xspectrum1d.py:253: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")
